# Building a single pandas dataframe with all eegs.

Run 02_single_eegs.py for the complete dataset.

Never concatenate dataframes inside a loop.

Keep a list of your chunks, and then concatenate everything as the final step.

In [2]:
import numpy as np
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import matplotlib.pyplot as plt

base_dir = "../../kaggle_data/hms"
# base_dir = "../../data/hms"
# base_dir = "/kaggle/input/hms-harmful-brain-activity-classification"

df_traincsv = pd.read_csv(f'{base_dir}/train.csv')

In [7]:
#
# Cleaning.
#

eegs_with_nans = np.load("../EDA/02_eegs_with_nan_ids.npy")

# Counting NaN in each eeg.
eegs_with_nans_qty = np.array([], dtype=int)
for eeg_id in eegs_with_nans:
    eeg = pd.read_parquet(f'{base_dir}/train_eegs/{eeg_id}.parquet')
    qty = len(eeg[pd.isna(eeg).any(axis=1)].index)
    eegs_with_nans_qty = np.append(eegs_with_nans_qty, qty)

df_qtys = pd.DataFrame({'eeg_id': eegs_with_nans, 'quantity': eegs_with_nans_qty})

# Discarding eeg_ids with more than 5 NaNs.
eeg_ids_gt5 = df_qtys.loc[df_qtys.quantity > 5].eeg_id
idxs_to_remove = df_traincsv.loc[df_traincsv.eeg_id.isin(eeg_ids_gt5.values)].index
idxs = df_traincsv.index.difference(idxs_to_remove)
eeg_ids_clean = df_traincsv.iloc[idxs].eeg_id.unique()
eeg_ids_clean = np.sort(eeg_ids_clean)


In [9]:
# This is just code for testing, using a sample. 
# Use 02_single_eegs.py for the complete dataset.

# Things to change when running on full dataset:
# - Name of output file.
# - Slice in eeg_ids_clean.
# - Remainder operation.

# df = pd.DataFrame()
thelist = []
for i, eeg_id in enumerate(eeg_ids_clean[10000:]):
    if i%500 == 0:
        print(f'{i} files loaded.')
    eeg = pd.read_parquet(f'{base_dir}/train_eegs/{eeg_id}.parquet')
    eeg = eeg.interpolate(limit_direction='both') # <<<<< Interpolation

    data = np.ones(eeg.shape[0], dtype=np.int16)*eeg_id
    eeg.insert(0, 'eeg_id', data)
    # df = pd.concat([df, eeg], ignore_index=True)
    thelist.append(eeg)

df = pd.concat(thelist, axis=0)
print('Saving to 02_single_eegs.parquet')
df.to_parquet('../data/02_single_eegs_10000_.parquet')

0 files loaded.
500 files loaded.
1000 files loaded.
1500 files loaded.
2000 files loaded.
2500 files loaded.
3000 files loaded.
3500 files loaded.
4000 files loaded.
4500 files loaded.
5000 files loaded.
5500 files loaded.
6000 files loaded.
6500 files loaded.
Saving to 02_single_eegs.parquet


In [10]:
del df

In [11]:
del thelist

In [13]:
import gc

In [14]:
gc.collect()

477

In [15]:
df1 = pd.read_parquet('../data/02_single_eegs_00000_09999.parquet')
df2 = pd.read_parquet('../data/02_single_eegs_10000_.parquet')


In [ ]:
df1 = pd.concat([df1,df2], axis=0)

In [4]:
df_reduced = pd.read_parquet("../data/02_single_eegs_reduced.parquet")
df_reduced

,eeg_id,Fp1,F3,C3,P3,F7,T3,T5,O1,Fz,...,Pz,Fp2,F4,C4,P4,F8,T4,T6,O2,EKG
0,1628180742,-80.519997,-70.540001,-80.110001,-108.750000,-120.330002,-88.620003,-101.750000,-104.489998,-99.129997,...,-97.040001,-77.989998,-88.830002,-112.120003,-108.110001,-95.949997,-98.360001,-121.730003,-106.449997,7.920000
1,1628180742,-80.449997,-70.330002,-81.760002,-107.669998,-120.769997,-90.820000,-104.260002,-99.730003,-99.070000,...,-96.019997,-84.500000,-84.989998,-115.610001,-103.860001,-97.470001,-89.290001,-115.500000,-102.059998,29.219999
2,1628180742,-80.209999,-75.870003,-82.050003,-106.010002,-117.500000,-87.489998,-99.589996,-96.820000,-119.680000,...,-91.110001,-99.440002,-104.589996,-127.529999,-113.349998,-95.870003,-96.019997,-123.879997,-105.790001,45.740002
3,1628180742,-84.709999,-75.339996,-87.480003,-108.970001,-121.410004,-94.750000,-105.370003,-100.279999,-113.839996,...,-95.040001,-99.230003,-101.220001,-125.769997,-111.889999,-97.459999,-97.180000,-128.940002,-109.889999,83.870003
4,1628180742,-90.570000,-80.790001,-93.000000,-113.870003,-129.960007,-102.860001,-118.599998,-101.099998,-107.660004,...,-98.510002,-95.300003,-88.930000,-115.639999,-99.800003,-97.500000,-88.730003,-114.849998,-100.250000,97.769997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11195,1529272612,17.570000,-11.800000,-39.880001,-6.070000,-13.520000,-14.650000,20.790001,25.379999,-4.830000,...,20.940001,-2.720000,-10.410000,23.270000,-14.090000,15.500000,-4.490000,-11.790000,31.900000,-48.290001
11196,1529272612,16.110001,-26.750000,-50.349998,-13.820000,-20.860001,-21.879999,13.510000,16.030001,-16.570000,...,14.350000,-14.400000,-20.190001,16.459999,-18.719999,8.910000,-9.690000,-15.910000,22.459999,31.570000
11197,1529272612,-19.090000,-37.900002,-51.820000,-20.620001,-17.930000,-17.700001,10.430000,4.990000,-38.820000,...,11.340000,-51.080002,-37.279999,18.090000,-18.059999,4.950000,-4.340000,-12.610000,-12.910000,22.080000
11198,1529272612,-10.810000,-30.510000,-55.419998,-22.059999,-24.250000,-23.900000,13.320000,14.190000,-23.420000,...,8.680000,-21.719999,-20.440001,18.000000,-20.950001,10.710000,-7.650000,-15.450000,11.790000,-54.020000
